<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llamaapi

In [4]:
import json
import pandas as pd
from llamaapi import LlamaAPI
from google.colab import userdata
import time
import requests

In [5]:
# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))

In [6]:
def evaluate_message(sender, message_body, index, total, max_retries=5):

    # Sanitize inputs
    sender = json.dumps(sender)
    message_body = json.dumps(message_body)

    # Display progress
    progress = (index / total) * 100  # calculate progress percentage
    print(f"Evaluating message {index}/{total} ({progress:.2f}%)")

    # Handle out-of-range float values in `message_body` and `sender`
    message_body = str(message_body)
    sender = str(sender)

    # Build the API request
    api_request_json = {
        "messages": [
            {"role": "user", "content": f"Is this message real or fake: '{message_body}' [Sender: {sender}]"}
        ],
        "functions": [
            {
                "name": "detect_fraudulent_sms",
                "description": "Analyze if an SMS message is legitimate or a scam",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sender": {
                            "type": "string",
                            "description": "The sender of the SMS",
                        },
                        "body": {
                            "type": "string",
                            "description": "The full text of the SMS message",
                        }
                    },
                    "required": ["sender", "body"],
                }
            }
        ],
        "stream": False,
        "function_call": "detect_fraudulent_sms",
        "function_parameters": {
            "sender": sender,
            "body": message_body
        }

    }

    retries=0

    while retries <= max_retries:
        try:
            response = llama.run(api_request_json)
            if response.status_code == 200:
                results = response.json()
                message_content = results['choices'][0]['message']['content']
                return "Real" if "real" in message_content.lower() else "Fake"
            else:
                print(f"HTTP Status Code: {response.status_code}")
                print("Response Content:", response.text)
                return "Undetermined"
        except json.JSONDecodeError as e:
            print(f"Attempt {retries+1} failed with JSONDecodeError: {str(e)}")
            time.sleep(2**retries)  # Exponential backoff
            retries += 1
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return "Undetermined"
    return "Undetermined"  # Use Undetermined if the API call was not successful

In [7]:
def update_excel_with_results(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path, header=None, names=['Sender', 'Body'])

    total = len(df)  # Total number of messages to process
    # Apply the evaluate_message function to each row with progress output
    results = [evaluate_message(row['Sender'], row['Body'], i+1, total) for i, row in df.iterrows()]
    df['Result'] = results

    # Save the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False, header=False)
    print("All messages have been evaluated and the results are saved back to the Excel file.")

In [8]:
# Example usage: specify the path to your Excel file that you want to update
update_excel_with_results("/content/smishTankSet.xlsx")

Evaluating message 1/1089 (0.09%)
Evaluating message 2/1089 (0.18%)
Evaluating message 3/1089 (0.28%)
Evaluating message 4/1089 (0.37%)
Evaluating message 5/1089 (0.46%)
Evaluating message 6/1089 (0.55%)
An error occurred: 'NoneType' object has no attribute 'lower'
Evaluating message 7/1089 (0.64%)
Evaluating message 8/1089 (0.73%)
Evaluating message 9/1089 (0.83%)
Evaluating message 10/1089 (0.92%)
Evaluating message 11/1089 (1.01%)
An error occurred: 'NoneType' object has no attribute 'lower'
Evaluating message 12/1089 (1.10%)
An error occurred: 'NoneType' object has no attribute 'lower'
Evaluating message 13/1089 (1.19%)
Evaluating message 14/1089 (1.29%)
An error occurred: 'NoneType' object has no attribute 'lower'
Evaluating message 15/1089 (1.38%)
An error occurred: 'NoneType' object has no attribute 'lower'
Evaluating message 16/1089 (1.47%)
Evaluating message 17/1089 (1.56%)
Evaluating message 18/1089 (1.65%)
Evaluating message 19/1089 (1.74%)
Evaluating message 20/1089 (1.84%)

KeyboardInterrupt: 